In [20]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the AnimalShelter class from the correct module
from animal_shelter import AnimalShelter

shelter = AnimalShelter()

# Read data from MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

app = JupyterDash('SimpleExample')

# Load the logo image
image_filename = 'picture.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
app.layout = html.Div(style={'backgroundColor': 'white'}, children=[  # Change background color to white
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '200px'})),  # Adjust image height
    html.Center(html.B(html.H1('Pravishna Nand CS 340 Dashboard'))),
    html.Center(html.P('Select up to five from the table for the map')),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WR'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DRIT'},
                {'label': 'Reset - returns unfiltered state', 'value': 'RESET'}
            ],
            value='RESET',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ])
])

@app.callback(
    [dash.Output('datatable-id', 'data'),
     dash.Output('datatable-id', 'columns')],
    [dash.Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'WR':
        df = pd.DataFrame(list(shelter.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Female'},
                {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        })))
    elif filter_type == 'MWR':
        df = pd.DataFrame(list(shelter.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Rottweiler'},
                    {'breed': 'Siberian Husky'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 26}},
                    {'age_upon_outcome_in_weeks': {'$lte': 156}}
                ]}
            ]
        })))
    elif filter_type == 'DRIT':
        df = pd.DataFrame(list(shelter.read({
            '$and': [
                {'sex_upon_outcome': 'Intact Male'},
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]},
                {'$and': [
                    {'age_upon_outcome_in_weeks': {'$gte': 20}},
                    {'age_upon_outcome_in_weeks': {'$lte': 300}}
                ]}
            ]
        })))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    return data, columns

@app.callback(
    dash.Output('map-id', 'children'),
    [dash.Input('datatable-id', 'selected_rows'),
     dash.Input('datatable-id', 'data')]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index and len(index) > 0 else 0

    latitude = dff.iloc[row]['location_lat']
    longitude = dff.iloc[row]['location_long']

    marker = dl.Marker(position=(latitude, longitude))

    return dl.Map(
        children=[
            dl.TileLayer(),
            marker
        ],
        style={'width': '1000px', 'height': '500px'},
        center=(latitude, longitude),
        zoom=10
    )
@app.callback(
    dash.Output('graph-id', 'children'),
    [dash.Input('datatable-id', 'selected_rows'),
     dash.Input('datatable-id', 'data')]
)
def update_graphs(selected_rows, data):
    # Add code to update the chart based on the selected data
    # For example, you can use Plotly Express to create a pie chart
    if selected_rows is None:
        return []

    df_view = pd.DataFrame(data)
    fig = px.pie(df_view, names='breed', title='Preferred Breeds')
    return dcc.Graph(figure=fig)

# Callback to highlight selected columns in the data table
@app.callback(
    dash.Output('datatable-id', 'style_data_conditional'),
    [dash.Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns:
        return [{'if': {'column_id': col}, 'background_color': '#D2F3FF'} for col in selected_columns]
    else:
        return []

if __name__ == '__main__':
    app.run_server(debug=True, port=8080)


Dash app running on http://127.0.0.1:8080/


/tmp/ipykernel_414817/3619742734.py:160: FutureWarning:

Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.

/tmp/ipykernel_414817/3619742734.py:160: FutureWarning:

Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.

/tmp/ipykernel_414817/3619742734.py:160: FutureWarning:

Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.

/tmp/ipykernel_414817/3619742734.py:160: FutureWarning:

Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.

